In [1]:
import pickle 
import pandas as pd
import numpy as np
import random
import time
import re

from keras.layers import Flatten,Dense,Dropout
from keras.models import Model
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
from context import pickles_path

In [3]:
from functions_tailored import select_N_random_races
from functions_tailored import build_train_validation_and_test_datasets

In [4]:
# load json and create model
json_file = open('CNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("CNN_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [7]:
dict_train_val_test = pickle.load(open(pickles_path+"dict_train_val_test.p", "rb" ))

In [8]:
[key for key in dict_train_val_test.keys()]

['train_labels',
 'val_labels',
 'test_labels',
 'X_train_raw',
 'X_val_raw',
 'X_test_raw',
 'X_train',
 'X_val',
 'X_test',
 'y_train',
 'y_val',
 'y_test']

In [10]:
dict_train_val_test['test_labels'][0]

'beagle_10206.jpg'

In [11]:
X_train = dict_train_val_test['X_train']
X_val = dict_train_val_test['X_val']
X_test = dict_train_val_test['X_test']
    
y_train = dict_train_val_test['y_train']
y_val = dict_train_val_test['y_val']
y_test = dict_train_val_test['y_test']

In [12]:
img = X_test[0]

In [13]:
X_test[0].shape

(224, 224, 3)

In [14]:
def predict_dog_race(img):
    import numpy as np
    import pandas as pd
    df_prediction = pd.DataFrame(columns=['prediction_score','index','race'])
    L_prediction = []
    
    if isinstance(img,np.ndarray):
        if img.shape==(224, 224, 3):
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
        elif img.shape==(1, 224, 224, 3):
            img = img
        else:
            return("BAD IMAGE SHAPE! SHAPE MUST BE (224, 224, 3) or (1, 224, 224, 3)")
        
        # load json and create model
        json_file = open('CNN_model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        # load weights into new model
        loaded_model.load_weights("CNN_model.h5")
        print("Loaded model from disk")
        prediction = loaded_model.predict(img)
        
        #Loading the vectorizer object : 
        vectorizer = pickle.load(open(pickles_path+"vectorizer_final_model.p", "rb" ))
        feature_names = vectorizer.get_feature_names()
        #Cleaning the features indexes :
        features_indexes = [int(re.sub('idx_','',key)) for key in feature_names]
        
        #Loading the label_encoder object :
        labelencoder = pickle.load(open(pickles_path+"label_encoder_final_model.p", "rb" ))
        true_labels = [labelencoder.inverse_transform(key) for key in features_indexes]
        
        #Mapping indexes, scores and races with pandas
        for index,score in enumerate(prediction[0]):
            dict_prediction_unsorted = {'prediction_score':score, 'index':index, 'race':labelencoder.inverse_transform(index)}
            df_prediction = df_prediction.append(dict_prediction_unsorted, ignore_index=True)
        
        #Sorting the final races by prediction scores :
        df_prediction.sort_values(by='prediction_score', ascending=False, inplace=True)
        for race, score in zip(df_prediction['race'], df_prediction['prediction_score']):
            L_prediction.append((race, score))
    
        print('Prediction over!')
        return(L_prediction)
    else:
        return("BAD PICTURE TYPE ! PICTURE MUST BE A numpy.ndarray")   

In [15]:
predict_dog_race([1,2,3])

'BAD PICTURE TYPE ! PICTURE MUST BE A numpy.ndarray'

In [37]:
dict_train_val_test['test_labels'][400]

'west_highland_white_terrier_118.jpg'

In [35]:
len(dict_train_val_test['X_train'])

982

In [38]:
img = X_test[400]
prediction = predict_dog_race(img)

Loaded model from disk
Prediction over!


c:\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will resul

In [39]:
prediction

[('west_highland_white_terrier', 0.9999986886978149),
 ('japanese_spaniel', 1.3568161421062541e-06),
 ('english_setter', 5.32590638258057e-12),
 ('labrador_retriever', 4.8056379236543667e-20),
 ('dhole', 1.1207705338870247e-22),
 ('bernese_mountain_dog', 3.730227003520254e-24),
 ('beagle', 3.088078289214875e-26),
 ('kelpie', 4.534546382647292e-30),
 ('rottweiler', 2.2913161415183026e-31),
 ('siberian_husky', 1.815002068983189e-31)]